<a href="https://colab.research.google.com/github/KhangTheKangaroo/Image-Depth/blob/main/Image_Depth_with_Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [2]:
!gdown --id 14gf8bcym_lTcvjZQmg8kwq3aXkENBxMQ
!unzip tsukuba.zip -d tsukuba

!gdown --id 1wxmiUdqMciuTOs0ouKEISl8-iTVXdOWn
!unzip Aloe_images.zip -d Aloe_images

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=14gf8bcym_lTcvjZQmg8kwq3aXkENBxMQ
To: /content/tsukuba.zip
100% 354k/354k [00:00<00:00, 74.3MB/s]
Archive:  tsukuba.zip
mapname:  conversion of  failed
 extracting: tsukuba/left.png        
 extracting: tsukuba/right.png       
 extracting: tsukuba/groundtruth.png  
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1wxmiUdqMciuTOs0ouKEISl8-iTVXdOWn
To: /content/Aloe_images.zip
100% 1.32M/1.32M [00:00<00:00, 71.2MB/s]
Archive:  Aloe_images.zip
  inflating: Aloe_images/Aloe/Aloe_left_1.png  
  inflati

In [3]:
def cosine_similarity (x, y): # Computes cosine similarity
  numerator = np.dot (x, y)
  denominator = np.linalg.norm(x) * np.linalg.norm(y)

  return numerator / denominator

In [11]:
def window_based_matching_cosine_similarity(left_img, right_img, disparity_range, kernel_size=5, save_result=True):

    # Read the images and convert them to grayscale
    left  = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    # Convert to float32
    left  = left.astype(np.float32)
    right = right.astype(np.float32)

    h, w = left.shape[:2] # Get height and width of left image


    disparity_map = np.zeros((h, w), np.uint8) # Matrix creation
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3

    for y in range(kernel_half, h - kernel_half):
        for x in range(kernel_half, w - kernel_half):

            # Find disparity where cost is the lowest
            disparity = 0
            cost_optimal  = -1

            for j in range(disparity_range):
                d = x - j
                cost = -1
                if (d - kernel_half) > 0:
                    wp = left[(y - kernel_half) : (y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    wqd = right[(y - kernel_half) : (y + kernel_half) + 1, (d - kernel_half):(d + kernel_half) + 1]

                    # Flatten to 1D
                    wp_flattened = wp.flatten()
                    wqd_flattened = wqd.flatten()

                    cost = cosine_similarity(wp_flattened, wqd_flattened)

                if cost > cost_optimal:
                    cost_optimal = cost
                    disparity = j


            disparity_map[y, x] = disparity * scale # Update matrix with disparity where cost is the lowest

    if save_result == True:
        cv2.imwrite('window_based_cosine_similarity.png', disparity_map)
        cv2.imwrite('window_based_cosine_similarity_color.png', cv2.applyColorMap(disparity_map, cv2.COLORMAP_JET))


    return disparity_map

In [ ]:
left_img_path = '/content/Aloe_images/Aloe/Aloe_left_1.png'
right_img_path = '/content/Aloe_images/Aloe/Aloe_right_1.png'
disparity_range = 64
kernel_size = 5


depth_cosine_similarity = window_based_matching_cosine_similarity(left_img_path, right_img_path, kernel_size = kernel_size, disparity_range = 64, save_result = True)

cv2_imshow(depth_cosine_similarity)
cv2_imshow(cv2.applyColorMap(depth_cosine_similarity, cv2.COLORMAP_JET))